In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import *
from scipy.signal import find_peaks
from sklearn.neighbors import KernelDensity
import librosa

In [2]:
def find_most_prominent_peak(peaks,peaks_prominences,valleys = None,valleys_prominences = None,min_threshold = 0,max_treshold = inf):
    max_prominence = 0
    
    max_peak = 0
    offset = (min_threshold + max_treshold) / 2
    for peak, prominence in zip(peaks, peaks_prominences):
        if peak > max_treshold:
            continue
        if peak > min_threshold and prominence * calc_prominence_factor(peak,offset=offset) > max_prominence:
            max_prominence = prominence * calc_prominence_factor(peak,offset=offset)
            max_peak = peak

    if valleys != None:
        for peak, prominence in zip(valleys, valleys_prominences):
            if peak > max_treshold:
                continue
            if peak > min_threshold and prominence * calc_prominence_factor(peak,offset=offset) > max_prominence:
                max_prominence = prominence * calc_prominence_factor(peak,offset=offset)
                max_peak = peak

    return (max_peak, max_prominence)
    
def calc_prominence_factor(x,threshold = 0.2,offset=0):
    return 1 # exp(0-100*(x-offset)**2)

In [14]:
class Analysis:
    def __init__(self,location):
        self.gyro = pd.read_csv(location + "/android.sensor.gyroscope.csv")
        self.accel = pd.read_csv(location + "/android.sensor.accelerometer.csv")
        self.beats = pd.read_csv(location + "/out.csv")
        # y, sr = librosa.load(location + "/cut.wav")
        # tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
        # # beats now contains the beat *frame positions*
        # # convert to timestamps like this:
        # self.libroa_beat_times = librosa.frames_to_time(beats, sr=sr)
    
    def preanalysis(self,prominence=(None,None)):

        self.accel_time = self.accel.iloc[:,0] / 1000000000
        self.gyro_time = self.gyro.iloc[:,0] / 1000000000

        self.accel_length = self.accel.iloc[:,1] ** 2 + self.accel.iloc[:,2] ** 2 + self.accel.iloc[:,3] ** 2

        self.accel_length_peaks_samples, temp = find_peaks(self.accel_length,prominence=prominence)
        self.accel_length_peaks = self.accel_time[self.accel_length_peaks_samples]
        self.accel_length_peaks_prominence = temp["prominences"]

        self.accel_peaks_x_samples , temp = find_peaks(self.accel.iloc[:,1],prominence=prominence)
        self.accel_peaks_x = self.accel_time[self.accel_peaks_x_samples]
        self.accel_peaks_x_prominence = temp["prominences"]

        self.accel_valleys_x_samples , temp = find_peaks(-1*self.accel.iloc[:,1],prominence=prominence)
        self.accel_valleys_x_prominence = temp["prominences"]
        self.accel_valleys_x = self.accel_time[self.accel_valleys_x_samples]

        self.accel_peaks_y_samples , temp = find_peaks(self.accel.iloc[:,2],prominence=prominence)
        self.accel_peaks_y_prominence = temp["prominences"]
        self.accel_peaks_y = self.accel_time[self.accel_peaks_y_samples]

        self.accel_valleys_y_samples , temp = find_peaks(-1*self.accel.iloc[:,2],prominence=prominence)
        self.accel_valleys_y_prominence = temp["prominences"]
        self.accel_valleys_y = self.accel_time[self.accel_valleys_y_samples]

        self.accel_peaks_z_samples , temp = find_peaks(self.accel.iloc[:,3],prominence=prominence)
        self.accel_peaks_z_prominence = temp["prominences"]
        self.accel_peaks_z = self.accel_time[self.accel_peaks_z_samples]

        self.accel_valleys_z_samples , temp = find_peaks(-1*self.accel.iloc[:,3],prominence=prominence)
        self.accel_valleys_z_prominence = temp["prominences"]
        self.accel_valleys_z = self.accel_time[self.accel_valleys_z_samples]

        self.gyro_peaks_x_samples , temp = find_peaks(self.gyro.iloc[:,1],prominence=prominence)
        self.gyro_peaks_x_prominence = temp["prominences"]
        self.gyro_peaks_x = self.gyro_time[self.gyro_peaks_x_samples]

        self.gyro_valleys_x_samples , temp = find_peaks(-1*self.gyro.iloc[:,1],prominence=prominence)
        self.gyro_valleys_x_prominence = temp["prominences"]
        self.gyro_valleys_x = self.gyro_time[self.gyro_valleys_x_samples]

        self.gyro_peaks_y_samples , temp = find_peaks(self.gyro.iloc[:,2],prominence=prominence)
        self.gyro_peaks_y_prominence = temp["prominences"]
        self.gyro_peaks_y = self.gyro_time[self.gyro_peaks_y_samples]

        self.gyro_valleys_y_samples , temp = find_peaks(-1*self.gyro.iloc[:,2],prominence=prominence)
        self.gyro_valleys_y_prominence = temp["prominences"]
        self.gyro_valleys_y = self.gyro_time[self.gyro_valleys_y_samples]

        self.gyro_peaks_z_samples , temp = find_peaks(self.gyro.iloc[:,3],prominence=prominence)
        self.gyro_peaks_z_prominence = temp["prominences"]
        self.gyro_peaks_z = self.gyro_time[self.gyro_peaks_z_samples]

        self.gyro_valleys_z_samples , temp = find_peaks(-1*self.gyro.iloc[:,3],prominence=prominence)
        self.gyro_valleys_z_prominence = temp["prominences"]
        self.gyro_valleys_z = self.gyro_time[self.gyro_valleys_z_samples]

    def analysis_data(self):
        min_threshold = 0
        max_threshold = inf

        self.beat_diff = (self.beats.iloc[:,0][0] + self.beats.iloc[:,0][len(self.beats.iloc[:,0])-1])/ (len(self.beats.iloc[:,0])-1)
        threshold = self.beat_diff / 2

        self.accel_peaks = []
        self.accel_peaks_axis = []
        self.accel_prominences = []
        gyro_peaks = []

        self.global_max_prominence = 0
        self.global_min_prominence = inf

        for beat, i in zip(self.beats.iloc[:,0],range(len(self.beats.iloc[:,0]))):

            #if i < (len(beats)-1):
            #    max_threshold = (beats[i+1] + beat) / 2
            #else: 
            #    max_threshold = inf

            min_threshold = beat - threshold
            max_threshold = beat + threshold

            accel_peaks_with_prominence = []
            gyro_peaks_with_prominence = []

            motion_beat, prominence = find_most_prominent_peak(self.accel_length_peaks,self.accel_length_peaks_prominence,max_treshold=max_threshold,min_threshold=min_threshold)
            if (motion_beat > 0):
                self.accel_peaks.append((motion_beat,prominence,1,beat))
                self.accel_prominences.append(prominence)
            else:
                self.accel_peaks.append((None,None,None,beat))
            # accel_peaks_with_prominence.append(find_most_prominent_peak(self.accel_peaks_x,self.accel_peaks_x_prominence,self.accel_valleys_x,self.accel_valleys_x_prominence,min_threshold, max_threshold))
            # accel_peaks_with_prominence.append(find_most_prominent_peak(self.accel_peaks_y,self.accel_peaks_y_prominence,self.accel_valleys_y,self.accel_valleys_y_prominence,min_threshold, max_threshold))
            # accel_peaks_with_prominence.append(find_most_prominent_peak(self.accel_peaks_z,self.accel_peaks_z_prominence,self.accel_valleys_z,self.accel_valleys_z_prominence,min_threshold, max_threshold))

            # gyro_peaks_with_prominence.append(find_most_prominent_peak(self.gyro_peaks_x,self.gyro_peaks_x_prominence,self.gyro_valleys_x,self.gyro_valleys_x_prominence,min_threshold, max_threshold))
            # gyro_peaks_with_prominence.append(find_most_prominent_peak(self.gyro_peaks_y,self.gyro_peaks_y_prominence,self.gyro_valleys_y,self.gyro_valleys_y_prominence,min_threshold, max_threshold))
            # gyro_peaks_with_prominence.append(find_most_prominent_peak(self.gyro_peaks_z,self.gyro_peaks_z_prominence,self.gyro_valleys_z,self.gyro_valleys_z_prominence,min_threshold, max_threshold))

            # max_accel_prominence = 0
            # max_peak_position = 0
            # for peak, prominence in accel_peaks_with_prominence:
            #     if prominence > max_accel_prominence:
            #         max_accel_prominence = prominence
            #         max_peak_position = peak

            # accel_peak_sum = 0
            # accel_peak_count = 0
            # axis = []


            
            # for (peak, prominence), index in zip(accel_peaks_with_prominence,range(3)):
            #     if prominence  >= (max_accel_prominence * 0.50) and abs(peak-max_peak_position) < threshold / 4:
            #         accel_peak_sum += peak
            #         accel_peak_count += 1
            #         axis.append(index)
            #         #self.accel_prominences.append(prominence)
            # if accel_peak_sum > 0:
            #     accel_peak = accel_peak_sum / accel_peak_count
            #     self.accel_peaks.append((accel_peak,max_accel_prominence,accel_peak_count,beat))
            #     self.accel_peaks_axis.append(axis)
            #     self.accel_prominences.append(max_accel_prominence)
            #     if max_accel_prominence > self.global_max_prominence:
            #         self.global_max_prominence = max_accel_prominence
            #     if max_accel_prominence < self.global_min_prominence:
            #         self.global_min_prominence = max_accel_prominence
            # else:
            #     self.accel_peaks.append((None,None,None,beat))
            #     self.accel_peaks_axis.append([])    
            min_threshold = max_threshold
        print(self.global_min_prominence, self.global_min_prominence)


    def plot_acceleration_preprosesed(self):
        plt.close()
        fig, axs = plt.subplots(ncols = 1, nrows = 1,sharey=True,sharex=True)

        fig.suptitle('acceleration with peaks with prominence > 25', fontsize=16)

        fig.supxlabel("time in seconds")
        fig.supylabel("acceleration in m/s^2")

        for x, y in zip(self.beats.iloc[:,0], self.beats.iloc[:,1]):

            seconds = x#+ offset_seconds

            if (y == 1):
                linestyle = 'solid'
            else:
                linestyle = 'dotted'
            axs.axvline(seconds, color = 'g',linestyle = linestyle)
            # axs[1].axvline(seconds, color = 'g',linestyle = linestyle)
            # axs[2].axvline(seconds, color = 'g',linestyle = linestyle)
            # axs[3].axvline(seconds, color = 'g',linestyle = linestyle)


        # axs[0].plot(self.accel_peaks_x, np.array(self.accel.iloc[self.accel_peaks_x_samples,1]), "x")
        # axs[0].plot(self.accel_valleys_x, np.array(self.accel.iloc[self.accel_valleys_x_samples,1]), "o")
        # axs[0].plot(self.accel_time,self.accel.iloc[:,1], color = 'r')
        # axs[0].grid(True)
        # axs[0].set_title("x")


        # axs[1].plot(self.accel_peaks_y, np.array(self.accel.iloc[self.accel_peaks_y_samples,2]), "x")
        # axs[1].plot(self.accel_valleys_y, np.array(self.accel.iloc[self.accel_valleys_y_samples,2]), "o")
        # axs[1].plot(self.accel_time,self.accel.iloc[:,2], color = 'r')
        # axs[1].grid(True)
        # axs[1].set_title("y")


        # axs[2].plot(self.accel_peaks_z, np.array(self.accel.iloc[self.accel_peaks_z_samples,3]), "x")
        # axs[2].plot(self.accel_valleys_z, np.array(self.accel.iloc[self.accel_valleys_z_samples,3]), "o")
        # axs[2].plot(self.accel_time,self.accel.iloc[:,3], color = 'r')
        # axs[2].grid(True)
        # axs[2].set_title("z")

        axs.plot(self.accel_length_peaks, np.array(self.accel_length)[self.accel_length_peaks_samples], "x")
        axs.plot(self.accel_time,self.accel_length, color = 'r')
        axs.grid(True)
        axs.set_title("length")


        plt.grid(True)
        plt.show()

    def plot_gyro_preprosesed(self):
        plt.close()
        fig, axs = plt.subplots(ncols = 1, nrows = 3,sharey=True,sharex=True)

        fig.suptitle('gyro with peaks with prominence > 25', fontsize=16)

        fig.supxlabel("time in seconds")
        fig.supylabel("gyro")

        for x, y in zip(self.beats.iloc[:,0], self.beats.iloc[:,1]):

            seconds = x#+ offset_seconds

            if (y == 1):
                linestyle = 'solid'
            else:
                linestyle = 'dotted'
            axs[0].axvline(seconds, color = 'g',linestyle = linestyle)
            axs[1].axvline(seconds, color = 'g',linestyle = linestyle)
            axs[2].axvline(seconds, color = 'g',linestyle = linestyle)


        axs[0].plot(self.gyro_peaks_x, np.array(self.gyro.iloc[self.gyro_peaks_x_samples,1]), "x")
        axs[0].plot(self.gyro_valleys_x, np.array(self.gyro.iloc[self.gyro_valleys_x_samples,1]), "o")
        axs[0].plot(self.gyro_time,self.gyro.iloc[:,1], color = 'r')
        axs[0].grid(True)
        axs[0].set_title("x")


        axs[1].plot(self.gyro_peaks_y, np.array(self.gyro.iloc[self.gyro_peaks_y_samples,2]), "x")
        axs[1].plot(self.gyro_valleys_y, np.array(self.gyro.iloc[self.gyro_valleys_y_samples,2]), "o")
        axs[1].plot(self.gyro_time,self.gyro.iloc[:,2], color = 'r')
        axs[1].grid(True)
        axs[1].set_title("y")


        axs[2].plot(self.gyro_peaks_z, np.array(self.gyro.iloc[self.gyro_peaks_z_samples,3]), "x")
        axs[2].plot(self.gyro_valleys_z, np.array(self.gyro.iloc[self.gyro_valleys_z_samples,3]), "o")
        axs[2].plot(self.gyro_time,self.gyro.iloc[:,3], color = 'r')
        axs[2].grid(True)
        axs[2].set_title("z")


        plt.grid(True)
        plt.show()

    def plot_beat_diff(self):
        self.calculate_running_average()
        fig, ax = plt.subplots()
        #plt.close()
        #plt.axhline(1)
        #plt.axhline(threshold)
        plt.xlabel("time in seconds")
        plt.ylabel("distance to next beat in beats")
        plt.title("time delta for each beat")
        plt.ylim([0, 0.5])


        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

        textstr = '\n'.join((r'%.4f ms' % (self.global_average  * 1000,),
                     r'%.4f beats' % (self.global_average/self.beat_diff,)))

        plt.text(0.05, 0.8,textstr, transform=ax.transAxes,bbox=props)

        #plt.axhline(-1*beat_diff)
        #plt.axhline(-1*threshold)
        label = True
        for peak, prominence , _ ,beat in self.accel_peaks:
            if (peak == None):
                continue
            diff = abs(peak - beat)
            if label:
                plt.plot(beat, diff/self.beat_diff,"bx", label="beat")
                label = False
            else:
                plt.plot(beat, diff/self.beat_diff,"bx") 


        for averages, beats in self.running_average:

            plt.plot(beats,averages/self.beat_diff,"r")

        plt.legend()
        plt.savefig("acceleration_used.pdf")
        plt.show()

    def plot_accelration_with_motion_beat(self):
        plt.close()
        fig, axs = plt.subplots(ncols = 1, nrows = 3,sharey=True,sharex=True)

        fig.suptitle('acceleration with motion- and music beats', fontsize=16)

        fig.supxlabel("time in seconds")
        fig.supylabel("acceleration in m/s^2")


        axs[2].axvline(self.beats.iloc[:,0][0], color = 'g',label = "music beat")


        for peak, axis in zip(self.accel_peaks, self.accel_peaks_axis):
            if 2 in axis:
                axs[2].axvline(peak[0],color='b',label = "motion beat")
                break
        axs[2].legend()
                
        #axs[accel_peaks_axis[0][0]].axvline(accel_peaks[0][0],color='b',label = "b")


        for x, y in zip(self.beats.iloc[:,0],self.beats.iloc[:,1]):
            seconds = x

            if (y == 1):
                linestyle = 'solid'
            else:
                linestyle = 'dotted'
            axs[0].axvline(seconds, color = 'g',linestyle = linestyle)
            axs[1].axvline(seconds, color = 'g',linestyle = linestyle)
            axs[2].axvline(seconds, color = 'g',linestyle = linestyle)

        for peak, axis in zip(self.accel_peaks, self.accel_peaks_axis):
            for i in axis:
                axs[i].axvline(peak[0], color = 'b')


        axs[0].plot(self.accel_peaks_x, np.array(self.accel.iloc[self.accel_peaks_x_samples,1]), "x")
        axs[0].plot(self.accel_valleys_x, np.array(self.accel.iloc[self.accel_valleys_x_samples,1]), "o")
        axs[0].plot(self.accel_time,self.accel.iloc[:,1], color = 'r')
        axs[0].grid(True)
        axs[0].set_title("x")


        axs[1].plot(self.accel_peaks_y, np.array(self.accel.iloc[self.accel_peaks_y_samples,2]), "x")
        axs[1].plot(self.accel_valleys_y, np.array(self.accel.iloc[self.accel_valleys_y_samples,2]), "o")
        axs[1].plot(self.accel_time,self.accel.iloc[:,2], color = 'r')
        axs[1].grid(True)
        axs[1].set_title("y")


        axs[2].plot(self.accel_peaks_z, np.array(self.accel.iloc[self.accel_peaks_z_samples,3]), "x")
        axs[2].plot(self.accel_valleys_z, np.array(self.accel.iloc[self.accel_valleys_z_samples,3]), "o")
        axs[2].plot(self.accel_time,self.accel.iloc[:,3], color = 'r')
        axs[2].grid(True)
        axs[2].set_title("z")


        plt.grid(True)
        axs[0].legend()
        plt.show()

    def analysis(self):
        self.preanalysis()
        self.analysis_data()

        
        new_array= np.reshape(self.accel_prominences, (1, -1))
        #print(new_array)

        kde = KernelDensity(kernel='epanechnikov',bandwidth='silverman').fit(np.array(self.accel_prominences).reshape((-1,1)))
        x_plot = np.linspace(floor(min(self.accel_prominences)),ceil(max(self.accel_prominences)),1000)[:,np.newaxis]
        log_dens = kde.score_samples(x_plot)

        peaks , _ = find_peaks(-1 * np.exp(log_dens),height=-0.02)
        min_prominence= x_plot[peaks[0]][0]

        min_prominence = 100

        print(min_prominence)

        self.preanalysis(prominence=(min_prominence,None))
        self.analysis_data()

        # plt.plot(x_plot[:,0],np.exp(log_dens))
        # plt.plot(x_plot[peaks],np.exp(log_dens)[peaks],"x")
        # plt.grid(True)
        # plt.show()
    

    def calculate_running_average(self,bandwith=10):
        averages = []
        beats = []
        elements = []
        self.running_average = []
        all_used_peaks = []
        temp = []
        for (peak, _ , _ ,beat), prominence in zip(self.accel_peaks, self.accel_prominences):
            if peak == None:
                elements = []
                if len(averages)>0:
                    self.running_average.append((averages,beats))
                    averages = []
                    beats = []
                    all_used_peaks.extend(temp)
                continue
            prominence_factor =1#  (min(prominence / (self.global_max_prominence * 0.5) , 1)) ** 100
            elements.append((abs(peak - beat),prominence_factor))
            temp.append((abs(peak - beat)))
            if len(elements) > bandwith:
                elements = elements[1:]
            if len(elements) > bandwith / 2:
                sum_prominence = 0
                count = 0
                for i, j in elements:                
                    sum_prominence += i 
                    count += 1
                average = sum_prominence / count
                averages.append(average)
                beats.append(beat)
        if len(averages)>0:
            all_used_peaks.extend(temp)
            self.running_average.append((averages,beats))
        self.global_average = sum(all_used_peaks) / len(all_used_peaks)

    def shift_beat(self, factor):
        print(self.beats)
        self.beats = self.beats + self.beat_diff*factor
        print(self.beats)



In [15]:
samstag = Analysis("crewcamp/samsstag/")
samstag.analysis()

inf inf
100
inf inf


In [16]:
samstag.plot_beat_diff()

In [19]:
samstag.plot_acceleration_preprosesed()

In [20]:
samstag.plot_accelration_with_motion_beat()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [12]:
y, sr = librosa.load("windows/clean.wav")
tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
# beats now contains the beat *frame positions*
# convert to timestamps like this:
beat_times = librosa.frames_to_time(beats, sr=sr)
beat_times

array([ 3.52943311,  3.90095238,  4.27247166,  4.64399093,  5.0155102 ,
        5.38702948,  5.78176871,  6.19972789,  6.59446712,  6.98920635,
        7.36072562,  7.80190476,  8.26630385,  8.73070295,  9.10222222,
        9.4737415 ,  9.86848073, 10.26321995, 10.65795918, 11.05269841,
       11.44743764, 11.84217687, 12.2369161 , 12.65487528, 13.02639456,
       13.39791383, 13.79265306, 14.18739229, 14.60535147, 15.04653061,
       15.4644898 , 15.90566893, 16.30040816, 16.69514739, 17.08988662,
       17.48462585, 17.85614512, 18.25088435, 18.62240363, 19.01714286,
       19.41188209, 19.80662132, 20.17814059, 20.57287982, 20.96761905,
       21.36235828, 21.73387755, 22.12861678, 22.50013605, 22.89487528,
       23.28961451, 23.66113379, 24.05587302, 24.42739229, 24.82213152,
       25.19365079, 25.58839002, 25.98312925, 26.37786848, 26.77260771,
       27.14412698, 27.53886621, 27.91038549, 28.30512472, 28.69986395,
       29.07138322, 29.46612245, 29.86086168, 30.23238095, 30.62

In [23]:
for beat in beat_times:
    plt.axvline(beat, color='g')
for beat in samstag.beats["time in seconds"]:
    plt.axvline(beat, color='r',alpha=0.5)

plt.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [19]:
def f(x):
    return np.exp(0-100*(x)**2)

x = np.linspace(-0.5,0.5,100)

plt.plot(x,f(x), color = "red")
plt.savefig("filter.pdf")
plt.show()